Using DollarDataset.txt dataset,
Please find top 5 greatest daily increase ( by percentage )

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

In [10]:
from pyspark import SparkContext

# import file
sc = SparkContext.getOrCreate()
currencyRDD = sc.textFile("DollarDataset.txt")

# drop empty lines
currencyRDD=currencyRDD.filter(lambda x: x[2].strip()!="")

# Split data with method
def split_data(line):
    info = line.split("\t")
    rate_date = info[1]
    rate = info[2].replace(".","")
    rate = rate.replace(",",".")
    return (rate_date, rate)

# split each line
closeRDD = currencyRDD.map(lambda line: split_data(line))

# drop empty rates
closeRDD = closeRDD.filter(lambda x: x[1] != '')
closeRDD = closeRDD.map(lambda line: (line[0],float(line[1])))

closeRDD = closeRDD.zipWithIndex()
closeRDD = closeRDD.map(lambda x : (x[1],x[0]))
# make openRDD the next day and join
openRDD = closeRDD.map(lambda x : (x[0]+1,x[1]))
changeRDD = openRDD.join(closeRDD)

changeRDD = changeRDD.map(lambda line :(line[1][1][0],line[1][0][1], line[1][1][1]))
changeRDD.take(10)

[('06-01-1950', 2.8, 2.8),
 ('12-01-1950', 2.8, 2.8),
 ('25-05-1950', 2.8, 2.8),
 ('31-05-1950', 2.8, 2.8),
 ('06-06-1950', 2.8, 2.8),
 ('12-06-1950', 2.8, 2.8),
 ('16-06-1950', 2.8, 2.8),
 ('22-06-1950', 2.8, 2.8),
 ('28-06-1950', 2.8, 2.8),
 ('04-07-1950', 2.8, 2.8)]

In [7]:
# count percentage method
def count_percentage(open_rate, close_rate):
    difference = close_rate - open_rate
    change = round((difference / open_rate) * 100, 2)
    return change

changeRDD = changeRDD.map(lambda line: (line[0], count_percentage(line[1], line[2])))

changeRDD.takeSample(False,10)

[('30-05-2017', 0.28),
 ('28-03-1980', 0.0),
 ('07-04-1993', 0.59),
 ('11-07-2005', -1.47),
 ('20-11-1956', 0.0),
 ('22-07-1983', -0.09),
 ('22-03-2017', -0.55),
 ('30-07-1993', 0.26),
 ('16-12-1960', 0.0),
 ('18-02-2015', 0.0)]

In [8]:
sortedRDD = changeRDD.sortBy(lambda line: line[1],ascending=False)
top5list = sortedRDD.take(5)

print("The top 5 daily increase : ", top5list)


The top 5 daily increase :  [('22-08-1960', 221.43), ('25-01-1980', 100.0), ('10-08-1970', 65.0), ('23-02-2001', 39.76), ('06-04-1994', 38.89)]
